VERİLERİN LİNEER REGRESYON İLE TAHMİN EDİLMESİ

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [5]:
df = pd.read_csv('data_cleaned.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   location      197 non-null    object
 1   city          197 non-null    object
 2   district      197 non-null    object
 3   neighborhood  197 non-null    object
 4   room          197 non-null    int64 
 5   living_room   197 non-null    int64 
 6   area          197 non-null    int64 
 7   age           197 non-null    int64 
 8   floor         197 non-null    int64 
 9   price         197 non-null    int64 
dtypes: int64(6), object(4)
memory usage: 15.5+ KB
None


In [8]:
df['city'] = df['city'].astype('category')
df['district'] = df['district'].astype('category')
df['neighborhood'] = df['neighborhood'].astype('category')
df['location'] = df['location'].astype('category')
df['room'] = df['room'].astype('int')
df['living_room'] = df['living_room'].astype('int')
df['area'] = df['area'].astype('int')
df['age'] = df['age'].astype('int')
df['floor'] = df['floor'].astype('int')
df['price'] = df['price'].astype('int')

print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   location      197 non-null    category
 1   city          197 non-null    category
 2   district      197 non-null    category
 3   neighborhood  197 non-null    category
 4   room          197 non-null    int64   
 5   living_room   197 non-null    int64   
 6   area          197 non-null    int64   
 7   age           197 non-null    int64   
 8   floor         197 non-null    int64   
 9   price         197 non-null    int64   
dtypes: category(4), int64(6)
memory usage: 15.5 KB
None


In [9]:
categorical_features = ['city', 'district', 'neighborhood']
numerical_features = ['room', 'living_room', 'area', 'age', 'floor']


In [10]:
full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])


In [11]:
X = df.drop('price', axis=1)
y = df['price']

20 ye 80 verileri ayırdım

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = Pipeline([
    ('preparation', full_pipeline),
    ('model', LinearRegression())
])

In [14]:
model.fit(X_train, y_train)

,steps,"[('preparation', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [15]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)


In [16]:
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")


MSE: 482649395406.92773
RMSE: 694729.7283166511
R2: 0.5920360488083516


R^2 0.59  geldi yani yüzde 59'luk başarılı. 50'nin altı kötü kabul edilir


In [17]:
feature_importances = model.named_steps['model'].coef_
print(feature_importances)


[ 1.19259121e+05  0.00000000e+00  6.52930864e+05 -3.36240455e+05
  1.26057260e+05  1.30570639e-08 -1.61537679e+05 -3.59301815e+05
  7.33149493e+04 -2.39691658e+04  4.71493711e+05  8.65488037e+04
 -4.20671936e+05 -1.42620879e+04  9.95601680e+05 -3.39023452e+04
 -3.18207484e+05  8.45326492e+05  5.64383856e+05 -1.47210519e+05
 -3.59301815e+05 -2.73916226e+05  1.08846451e+06  6.54994111e+05
  6.69062650e+05 -5.00593479e+05 -2.45062005e+05  2.88821805e+05
  5.78278190e+05 -8.71008389e+05 -1.55243880e+05  1.45340666e+05
 -3.10532586e+05 -2.70551517e+05 -3.50329236e+05 -1.61537679e+05
 -3.77579374e+05  4.28398509e+05  1.56591956e+05  9.48806579e+04
  9.21390357e+03  3.85364068e+05  3.53751911e+05  7.79167185e+04
 -7.80198831e+05 -4.20818239e+05 -5.38186737e+05  4.61902800e+05
  1.46004067e+05  1.25611745e+05  1.22584933e+06 -1.53775928e+06
 -1.66548443e+06 -2.94122652e+05 -8.83422342e+04  4.68814745e+05
 -2.49197797e+05  2.32725202e+05  3.00172379e+05]


In [18]:
print("Numerical Features")
for i in range(len(numerical_features)):
    print(numerical_features[i], feature_importances[i])


Numerical Features
room 119259.12115402083
living_room 0.0
area 652930.8644229042
age -336240.4547382257
floor 126057.25983994496


Yukarıdaki değerler negatifse ters orantı var demektir

In [19]:
print("Categorical Features")
for i in range(len(categorical_features)):
    for j in range(len(model.named_steps['preparation'].transformers_[1][1].categories_[i])):
        print(
            model.named_steps['preparation'].transformers_[1][1].categories_[i][j],
            feature_importances[len(numerical_features) + j]
        )


Categorical Features
afyonkarahisar 1.305706390395959e-08
bolvadin 1.305706390395959e-08
cay -161537.6793182501
dinar -359301.81530136126
ihsaniye 73314.94925963992
merkez -23969.165769562842
ali_ihsan_pasa 1.305706390395959e-08
battalgazi -161537.6793182501
beyazit -359301.81530136126
burmali 73314.94925963992
camikebir -23969.165769562842
cumhuriyet 471493.71112955996
dervis_pasa 86548.80365621673
dortyol -420671.9358200391
dumlupinar -14262.087864211579
elbiz 995601.6802278991
erenler -33902.34524306451
erkmen_cumhuriyet -318207.4839183987
erkmen_fevzi_cakmak 845326.4915175042
erkmen_hurriyet 564383.8562479496
esentepe -147210.51864534337
esrefpasa -359301.81530136126
fatih -273916.2259756465
gazi 1088464.5082659996
gazi_mehmet_cavus 654994.111320298
gazligol_yunus_emre 669062.6503214807
guvenevler -500593.47910363675
hamidiye -245062.00494506827
hattat_karahisar 288821.8050314139
hoca_ahmet_yesevi 578278.189888217
ihsaniye -871008.3885256379
istiklal -155243.8795184811
kanlica 1453

ÖRNEK DENEME   YÜZDE 59 BAŞARI İÇİN

In [21]:
new_data = pd.DataFrame({
    'city': ['afyonkarahisar'],
    'district': ['merkez'],
    'neighborhood': ['esentepe'],
    'room': [2],
    'living_room': [1],
    'area': [100],
    'age': [10],
    'floor': [3]
})
print(model.predict(new_data))

[2234050.17748244]
